# 第8章 機械学習の適用1 - 感情分析

* https://github.com/rasbt/python-machine-learning-book/blob/master/code/ch08/ch08.ipynb
* 自然言語処理(Natural Language Processing: NLP)
  * 感情(センチメント)分析(sentiment analysis)
* 極性(polarity)

## 8.1 IMDbの映画レビューデータセットの取得

* 意見マイニング(opinion mining)
* IMDb(Internet Movie Database)
* http://ai.stanford.edu/~amaas/data/sentiment/
  * http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
* 以下のコマンドでダウンロード
  
```sh
$ curl -O http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
$ tar xfz aclImdb_v1.tar.gz
```

* 進行状況を見るために pyprind をインストール

```sh
$ pip install pyprind
```

In [1]:
# データを読み込む
import pyprind
import pandas as pd
import os
pbar = pyprind.ProgBar(50000)
labels = {'pos': 1, 'neg': 0}
df = pd.DataFrame()
for set in ('test', 'train'):
    for label in ('pos', 'neg'):
        path = os.path.join(('.', 'aclImdb', set, label))
        for file in os.listdir(path):
            with open(os.path.join(path, file), encoding='utf-8') as f:
                txt = f.read()
                df = df.append([[txt, labels[label]]], ignore_index=True)
                pbar.update()
df.columns = ['review', 'sentiment']

ImportError: No module named 'pyprind'

In [ ]:
import numpy as np
np.random.seed(0)
# 行の順番をサッフル
df = df.reindex(np.random.permutation(df.index))
# CSV に保存
df.to_csv('./movie_data.csv', index=False)

In [ ]:
df = pd.read_csv('./movie_dat.csv')
df.head(3)

## 8.2 BoWモデルの紹介

* BoW(Bag-of-Words)

1. ドキュメントの集合全体から、たとえば単語という一意なトークン(token)からなる語彙(vocabulary)を作成する
2. 各ドキュメントでの各単語の出現回数を含んだ特徴ベクトルを構築する

* 疎ベクトル(sparse vector)

### 8.2.1 単語を特徴ベクトルに変換する

* 生の出現頻度(raw term frequencies): tf(t, d)

In [4]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [5]:
# 語彙の中身を出力
print(count.vocabulary_)
# 特徴ベクトルを出力
print(bag.toarray())

{'is': 1, 'weather': 8, 'sweet': 5, 'shining': 3, 'the': 6, 'one': 2, 'sun': 4, 'and': 0, 'two': 7}
[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


### 8.2.2 TF-IDFを使って単語の関連性を評価する

* TF-IDF(Term Frequency-Inverse Document Frequency)
* TF: 単語の出現頻度
* IDF: 逆文書頻度

In [6]:
np.set_printoptions(precision=2)

from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.    0.43  0.    0.56  0.56  0.    0.43  0.    0.  ]
 [ 0.    0.43  0.    0.    0.    0.56  0.43  0.    0.56]
 [ 0.5   0.45  0.5   0.19  0.19  0.19  0.3   0.25  0.19]]
